In [1]:
from sentence_transformers import SentenceTransformer
from datetime import datetime

import logging
import re

import numpy as np
import pandas as pd
import hdbscan
from sklearn.cluster import AgglomerativeClustering
import umap

import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
model_path = 'distiluse-base-multilingual-cased-v1' # kpfsbert  학습해서 대체 예정.
cluster_mode = 'title'

In [4]:
# 데이터로딩
DATA_PATH = 'kpfSBERT_Clustering\data/newstrust_20210601_samlple.json'
df = pd.read_json(DATA_PATH, encoding='utf-8')
df.head()
len(df)

12006

In [5]:
#sBERT 모델 로딩
model = SentenceTransformer(model_path) 

In [6]:
# 차원축소 -> 연산량을 줄이기 위한 선택영역, 문장->문서로 넘어가면 차원축소 필수 있수도?

# UMAP 차원축소 실행
def umap_process(corpus_embeddings, n_components=5):
    umap_embeddings = umap.UMAP(n_neighbors=15, 
                                n_components=n_components, 
                                metric='cosine').fit_transform(corpus_embeddings)
    return umap_embeddings


In [7]:
def hdbscan_process(corpus, corpus_embeddings, min_cluster_size=15, min_samples=10, umap=True, n_components=5, method='eom'):
    
    if umap:
        umap_embeddings = umap_process(corpus_embeddings, n_components)
    else:
        umap_embeddings = corpus_embeddings
    
    cluster = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                              min_samples=10,
                              allow_single_cluster=True,
                              metric='euclidean',
                              core_dist_n_jobs=1,# knn_data = Parallel(n_jobs=self.n_jobs, max_nbytes=None) in joblib
                              cluster_selection_method=method).fit(umap_embeddings) #eom leaf

    docs_df = pd.DataFrame(corpus, columns=["Doc"])
    docs_df['Topic'] = cluster.labels_
    docs_df['Doc_ID'] = range(len(docs_df))
    docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})
    
    return docs_df, docs_per_topic

In [8]:
# 카테고리별 클러스터링
start = datetime.now()
print('작업 시작시간 : ', start)
previous = start
bt_prev = start

tot_df = pd.DataFrame()

print(' processing start... with cluster_mode :', cluster_mode)

category = df.category.unique()

df_category = []
for categ in category:
    df_category.append(df[df.category==categ])
cnt = 0
rslt = []
topics = []
#순환하며 데이터 만들어 df에 고쳐보자
for idx, dt in enumerate(df_category):

    corpus = dt[cluster_mode].values.tolist()
    # '[보통공통된꼭지제목]' 형태를 제거해서 클러스터링시 품질을 높인다.
    for i, cp in enumerate(corpus):
        corpus[i] = re.sub(r'\[(.*?)\]', '', cp)
#     print(corpus[:10])
    corpus_embeddings = model.encode(corpus, show_progress_bar=True)

    docs_df, docs_per_topic = hdbscan_process(corpus, corpus_embeddings,
                                               umap=False, n_components=15, #연산량 줄이기 위해 umap 사용시 True 
                                               method='leaf',
                                               min_cluster_size=5,
                                               min_samples=30,
                                               )
    cnt += len(docs_df)

    rslt.append(docs_df)
    topics.append(docs_per_topic)
    dt['cluster'] = docs_df['Topic'].values.tolist()
    tot_df = pd.concat([tot_df,dt])

    bt = datetime.now()
    print(len(docs_df), 'docs,', len(docs_per_topic)-1 ,'clusters in', category[idx], ', 소요시간 :', bt - bt_prev)
    bt_prev = bt
now = datetime.now()
print(' Total docs :', cnt,'in', len(rslt), 'Categories', ', 소요시간 :', now - previous)
previous = now

#cluster update    

df['cluster'] = tot_df['cluster'].astype(str)
    
end = datetime.now()
print('작업 종료시간 : ', end, ', 총 소요시간 :', end - start)

작업 시작시간 :  2022-08-05 11:12:16.137773
 processing start... with cluster_mode : title


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

819 docs, 8 clusters in IT 과학 , 소요시간 : 0:00:42.184294


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

2662 docs, 19 clusters in 경제 , 소요시간 : 0:02:07.924729


...

259 docs, 2 clusters in 교육 , 소요시간 : 0:00:11.782991


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

79 docs, 0 clusters in 라이프스타일 , 소요시간 : 0:00:03.947092


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

266 docs, 2 clusters in 스포츠 , 소요시간 : 0:00:11.965528
 Total docs : 12006 in 11 Categories , 소요시간 : 0:09:47.836986
작업 종료시간 :  2022-08-05 11:22:03.994390 , 총 소요시간 : 0:09:47.856617


In [37]:
categ = '사회'
condition = (df.category == categ) & (df.cluster == '-1') # 조건식 작성b

In [38]:
test = df[condition]
print(len(test))
test.title.values.tolist()

4343


['기보?창원산업진흥원, 기술이전・기술금융 지원 업무협약 체결',
 "용인시, 시청에서 일회용품 안쓰는 'One-Zero day' 운영",
 '생산성본부・가평군, 소상공인 경영환경개선 지원 나선다',
 "정부, '지방소멸 위기' 대응 위한 기반 마련…행정・재정 지원 추진",
 '중기부, 지역가치 창업가 협업지원 추진',
 '동반위-국민은행, 제3회 동반성장 대기업 협력사 PR챌린지 개최',
 "농협은행, 10일 '생애주기별 자산관리 웹세미나' 개최",
 '[부음]한홍수(포항대 총장)씨 장인상',
 "광주경제고용진흥원, 생활 속 일회용품 줄이기 '고고 챌린지' 참여",
 '화학연, 탄소중립 위한 친환경 산업가스 개발 업무 협약 2건 체결',
 "당근마켓 '내 근처'에서 만나는 청소연구소…청소 예약・결제 한 번에",
 "SK렌터카, 제주지점 공항 인근으로 이전...셔틀 거리 '10→2분'",
 "야놀자, 지역경제 활성화 위한 '전국 1주' 프로젝트 진행",
 '[인사]NEWS 더원',
 "NH농협생명, '무배당 두개만묻는NH건강보험(갱신형)' 출시",
 '[인사]ABL생명',
 'DTG 분석해 민자도로 맞춤형 안전대책 수립',
 '[부음]조선(부산일보 상임감사)씨 부친상',
 '동서식품, 맥심 커피믹스 종이 손잡이 도입...플라스틱 저감 동참',
 "광주시, 'K-Pop 스타의거리' 콘텐츠 개발한다…대표 랜드마크 마케팅",
 "환경부 내 '기후탄소정책실' '물관리정책실' 신설",
 '[부음]안재영(ubc울산방송국 부장)씨 부친상',
 '[부음]윤동섭(연세의료원장)씨 장인상',
 "신세계아이앤씨, 청소년 해커톤 대회 '헬로 뉴( ) 월드' 참가자 모집",
 '[부음]이선영(SR 경영지원실장) 씨 장인상',
 '전남테크노파크, 광양만권경제자유구역 비즈니스 역량강화 참여기업 모집',
 '[인사]기상청',
 "철도연, '스크롤 공기압축기 상용화기술' 이전...철도차량・도시버스 제동장치 고장",
 '공공폐자원관리시설, 설치지역 주민과 운영이익 나눈다',
 "경동나비엔 '키

In [13]:
# 클러스터별 주제어 추출확인
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [31]:
category_id = 1

In [32]:
tf_idf, count = c_tf_idf(topics[category_id].Doc.values, m=len(corpus))

top_n_words = extract_top_n_words_per_topic(tf_idf, count, topics[category_id], n=20)
topic_sizes = extract_topic_sizes(rslt[category_id]); topic_sizes.head(10)

,Topic,Size
0,-1,2323
18,17,94
19,18,31
10,9,20
17,16,19
5,4,18
1,0,18
12,11,16
2,1,15
15,14,14


In [33]:
top_n_words[2][:10]

[('이마트', 0.36454569064307263),
 ('바이오퍼블릭', 0.3225020696614742),
 ('브랜드', 0.2771173240786722),
 ('건기식', 0.2687517247178952),
 ('자체', 0.18924087601655232),
 ('론칭', 0.17462114061749098),
 ('오비맥주', 0.15623386741845968),
 ('수제맥주', 0.15623386741845968),
 ('biopublic', 0.15524917867370647),
 ('kbc', 0.11547892570559953)]